In [ ]:
!pip install py7zr
from py7zr import unpack_7zarchive
import shutil
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

# unzitensorflow.keras.preprocessingshutil.unpack_archive('../input/cifar-10/train.7z')




In [ ]:
shutil.unpack_archive('../input/cifar-10/test.7z')
shutil.unpack_archive('../input/cifar-10/train.7z')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import os
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import load_img ,ImageDataGenerator
from tensorflow.keras.models import Model ,Sequential 
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense,SeparableConv2D ,Flatten

# load data

In [ ]:
train_dir=os.listdir('./train')
train_dir_len = len(train_dir)
print('Length :\t',
      train_dir_len)

In [ ]:
train_labels = pd.read_csv('../input/cifar-10/trainLabels.csv')
train_images = pd.DataFrame(columns=['id','labels','path'] ,dtype =str)
test_labels =pd.read_csv('../input/cifar-10/sampleSubmission.csv')

In [ ]:
test_labels

In [ ]:
tf.random.set_seed(
    32
)

In [ ]:
train_root = './train/'
for i in range( 0 , train_dir_len):
    path = train_root+str(i+1)+'.png'
    if os.path.exists(path):
        
        train_images=train_images.append([
         {   'id' :train_labels['id'].iloc[i],
            'label' :train_labels['label'].iloc[i],
            'path' :str(i+1)+'.png'}
        ])

In [ ]:
train_images=train_images.sort_values(by=['id'])

train_images.head(10)
train_images.drop(['labels'] ,axis=1)

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for name in class_names :
    index = class_names.index(name)
    train_images.loc[train_images['label'] ==name ,'label'] =str(index)
display_groupby = train_images.groupby(['label']).count()
display_groupby.head(10)

In [ ]:
train_images=train_images.drop(['labels'] ,axis=1)

In [ ]:
batch_size = 64 
num_class= 10


In [ ]:
train_labels['id'] = train_labels['id'].astype(str)

In [ ]:
train_images

In [ ]:
train_l = train_labels[:35000]

In [ ]:
val_l = train_labels[35000:]

In [ ]:
data_generator = ImageDataGenerator(rescale=1/255. ,rotation_range=True , validation_split=.3)

In [ ]:
train_generator =  data_generator.flow_from_dataframe(dataframe=train_images,directory='./train',
                                                    x_col='path',
                                                    y_col='label',
                                                    subset='training',
                                                    batch_size=batch_size,
                                                    
                                                    target_size=(32,32),
                                                    class_mode='categorical')

In [ ]:
val_generator = data_generator.flow_from_dataframe(dataframe=train_images,directory='./train',
                                                    x_col='path',
                                                    y_col='label',
                                                    subset='validation',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    target_size=(32,32),
                                                    class_mode='categorical')

In [ ]:
model = Sequential()
model.add(SeparableConv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(SeparableConv2D(32, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.2))

model.add(SeparableConv2D(64, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(SeparableConv2D(64, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.3))

model.add(SeparableConv2D(128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(SeparableConv2D(128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())

model.add(MaxPooling2D())
model.add(Dropout(0.4))



model.add(SeparableConv2D(128,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(SeparableConv2D(128,kernel_size=(1,1),strides=(1,1)
                 ,padding='valid',activation='relu'))
model.add(BatchNormalization())
# model.add(MaxPooling2D())
model.add(Dropout(0.6))
model.add(SeparableConv2D(10,kernel_size=(1,1),strides=(1,1),padding='valid',activation='softmax'))

model.add(Flatten())

# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))
# model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

In [ ]:
reduce_le = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5,factor=.7 , min_denta=.00001)
se = keras.callbacks.EarlyStopping(monitor="val_loss" ,patience=10 ,verbose=1 ,restore_best_weights=True)
history = model.fit(train_generator,epochs=100,validation_data=val_generator ,callbacks=[reduce_le,se] )

In [ ]:
_, ax = plt.subplots(2, 1)
ax[0].plot(history.history['loss'], color='b', label='loss')
ax[0].plot(history.history['val_loss'], color='r', label='val_loss')
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label='accuracy')
ax[1].plot(history.history['val_accuracy'], color='r', label='val_accuracy')
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
!mkdir ./data/
!mv ./test ./data

In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255.)
test_generator = test_data_generator.flow_from_directory(directory='./data',
            batch_size=batch_size,
            shuffle=False,color_mode='rgb',
            target_size=(32,32),
            class_mode=None)

In [ ]:
train_pre = model.predict(train_generator)


In [ ]:
val_pre = model.predict(val_generator)


In [ ]:
prediction = model.predict(test_generator)
pre_data = pd.DataFrame(data=prediction)

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for name in class_names :
    index = class_names.index(name)
    train_l.loc[train_l['label'] ==name ,'label'] =str(index)
    val_l.loc[val_l['label'] ==name ,'label'] =str(index)

display_groupby = train_l.groupby(['label']).count()
display_groupby.head(10)


In [ ]:
# display_groupby = train_labels.groupby(['label']).count()
# display_groupby


In [ ]:
# train_labels['label']=train_labels['label'].astype('int32')


In [ ]:
# test_lab['label']=test_labels['label'].astype('int32')

In [ ]:
train_l

In [ ]:
 train_l=train_l.drop(['id'] ,axis=1)

In [ ]:
 val_l=val_l.drop(['id'] ,axis=1)

In [ ]:
val_l =val_l.astype('int64')
train_l =train_l.astype('int64')

In [ ]:
# val_l =val_l*255
# train_l =train_l*255

In [ ]:
val_l.head(100)

In [ ]:
pre_data

In [ ]:
from xgboost import XGBClassifier

xgbmodel = XGBClassifier(objective='multi:softprob', 
                      num_class= 10,use_label_encode=False)
xgbmodel.fit(train_pre, train_l)
xgbmodel.score(val_pre, val_l)

In [ ]:



prediction =xgbmodel.predict(pre_data)

In [ ]:
# prediction = np.argmax(prediction, axis=1)
submission = pd.DataFrame(columns=['id', 'label'], dtype=str)
submission['label'] = [class_names[int(i)] for i in prediction]
submission['id'] = [ (''.join(filter(str.isdigit, name ))) for name in test_generator.filenames]
# submission = pd.DataFrame({'id':test_labels['id'],'label':prediction})
submission.sort_values(by=['id'])
submission.head()

In [ ]:
prediction


In [ ]:
%rm -rf ./train
%rm -rf ./data

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
prediction

In [ ]:
test_labels